Copyright 2022 Google LLC. SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Experiment: Reactive Controllers on Toy Tasks

This notebook is a part of the open-source code release associated with the paper:

[Code as Policies: Language Model Programs for Embodied Control](https://code-as-policies.github.io/)

This notebook gives the results corresponding to Seciont IV.D in the paper which generates reactive controllers for toy tasks.

1) Please obtain an OpenAI API Key here:
https://openai.com/blog/openai-api/

2) Gain Codex access by joining the waitlist here:
https://openai.com/blog/openai-codex/

Once you have Codex access you can use `code-davinci-002` as the `model_name`. Using the GPT-3 model (`text-dainvci-002`) is also ok, but performance won't be as good (there will be more code logic errors).


In [ ]:
openai_api_key = 'YOUR KEY HERE'
model_name = 'code-davinci-002' # 'text-davinci-002'

# Setup

In [ ]:
! pip install numpy scipy shapely openai pygments > /dev/null 2>&1

import openai
openai.api_key = openai_api_key

In [ ]:
from copy import copy
from time import sleep
from tqdm.auto import trange

import ast
import astunparse

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter

def exec_safe(code_str, gvars, lvars):
  banned_phrases = ['import', '__']
  for phrase in banned_phrases:
    assert phrase not in code_str
  
  empty_fn = lambda *args, **kwargs: None
  custom_gvars = merge_dicts([
      gvars,
      {'exec': empty_fn, 'eval': empty_fn}
  ])
  exec(code_str, custom_gvars, lvars)

default_query_kwargs = {
    'engine': model_name,
    'max_tokens': 512,
    'temperature': 0,
}

def lmp(base_prompt, query, stop_tokens=None, log=True, return_response=False, query_kwargs=None):
    new_prompt = f'{base_prompt}\n{query}'
    use_query_kwargs = copy(default_query_kwargs)
    if query_kwargs is not None:
      use_query_kwargs.update(query_kwargs)
    response = openai.Completion.create(
        prompt=new_prompt, stop=stop_tokens, **use_query_kwargs
    )['choices'][0]['text'].strip()

    if log:
      print(query)
      print(response)

    if return_response:
      return response

def lmp_fgen(prompt, f_name, f_sig, stop_tokens=['# define function:', '# example:'], recurse=False, 
             context_vars=None, bug_fix=False, log=True, return_src=False, query_kwargs=None, info=''):
    query = f'# define function: {f_sig}.'
    if info:
      query = f'{query}\n# info: {info}.'
    f_src = lmp(prompt, query, stop_tokens=stop_tokens, log=False, return_response=True, query_kwargs=query_kwargs)
    if bug_fix:
        f_src = openai.Edit.create(
          model='code-davinci-edit-001',
          input='# ' + f_src,
          temperature=0,
          instruction="Fix the bug if there is one. Improve readability. Keep same inputs and outputs. Only small changes. No comments.",
        )['choices'][0]['text'].strip()

    if context_vars is None:
        context_vars = {}
    gvars = context_vars
    lvars = {}

    f_success = True
    try:
      exec_safe(f_src, gvars, lvars)
      f = lvars[f_name]
    except Exception as e:
      print(e)
      f = lambda *args, **kargs: None   
      f_success = False 

    if recurse and f_success:
      # recursively define child_fs in the function body if needed
      f_def_body = astunparse.unparse(ast.parse(f_src).body[0].body)
      potential_child_fs, potential_child_f_sigs = {}, {}
      f_parser = FunctionParser(potential_child_fs, potential_child_f_sigs)
      f_parser.visit(ast.parse(f_def_body))
      for potential_child_f_name, potential_child_f_sig in potential_child_f_sigs.items():
        if potential_child_f_name in potential_child_fs:
          potential_child_fs[potential_child_f_name] = potential_child_f_sig

      child_fs, child_f_srcs = {}, {}
      for child_f_name, child_f_sig in potential_child_fs.items():
        all_vars = merge_dicts([context_vars, child_fs])
        if not var_exists(child_f_name, all_vars):
          child_f, child_f_src = lmp_fgen(
              prompt, child_f_name, child_f_sig, 
              stop_tokens=stop_tokens, 
              context_vars=all_vars, 
              bug_fix=bug_fix,
              log=False, 
              recurse=True,
              return_src=True,
              query_kwargs=query_kwargs
            )

          child_fs[child_f_name] = child_f
          child_f_srcs[child_f_name] = child_f_src

      if len(child_fs) > 0:
        # redefine parent f so newly created child_fs are in scope
        gvars = merge_dicts([context_vars, child_fs])
        lvars = {}
      
        exec_safe(f_src, gvars, lvars)
        
        f = lvars[f_name]

    if log:
        to_print = highlight(f'{query}\n{f_src}', PythonLexer(), TerminalFormatter())
        print(f'LMP FGEN created:\n\n{to_print}\n')

    if return_src:
        return f, f_src
    return f

class FunctionParser(ast.NodeTransformer):

    def __init__(self, fs, f_assigns):
      super().__init__()
      self._fs = fs
      self._f_assigns = f_assigns

    def visit_Call(self, node):
        self.generic_visit(node)
        if isinstance(node.func, ast.Name):
            f_sig = astunparse.unparse(node).strip()
            f_name = astunparse.unparse(node.func).strip()
            self._fs[f_name] = f_sig
        return node

    def visit_Assign(self, node):
        self.generic_visit(node)
        if isinstance(node.value, ast.Call):
            assign_str = astunparse.unparse(node).strip()
            f_name = astunparse.unparse(node.value.func).strip()
            self._f_assigns[f_name] = assign_str
        return node

def var_exists(name, all_vars):
    try:
        eval(name, all_vars)
    except:
        exists = False
    else:
        exists = True
    return exists

def merge_dicts(dicts):
    return {
        k : v 
        for d in dicts
        for k, v in d.items()
    }

In [ ]:
prompt_f_gen = '''
import numpy as np
from shapely.geometry import *
from shapely.affinity import *
from utils import get_obj_outer_pts_np

# define function: total = get_total(xs=numbers).
def get_total(xs):
    return np.sum(xs)

# define function: y = eval_line(x, slope, y_intercept=0).
def eval_line(x, slope, y_intercept):
    return x * slope + y_intercept

# define function: pt_np = get_pt_to_the_left(pt_np, dist).
def get_pt_to_the_left(pt_np, dist):
    delta = np.array([-dist, 0])
    return translate_pt_np(pt_np, delta=delta)

# define function: pt_np = get_pt_to_the_top(pt_np, dist).
def get_pt_to_the_top(pt_np, dist):
    delta = np.array([0, dist])
    return translate_pt_np(pt_np, delta=delta)

# define function line = make_line_by_length(length=x).
def make_line_by_length(length):
  start = np.array([0, 0])
  end = np.array([length, 0])
  line = make_line(start=start, end=end)
  return line

# define function: line = make_vertical_line_by_length(length=x).
def make_vertical_line_by_length(length):
  line = make_line_by_length(length)
  vertical_line = rotate(line, 90)
  return vertical_line

# define function: pt = interpolate_line(line, t=0.5).
def interpolate_line(line, t):
  pt = line.interpolate(t, normalized=True)
  return np.array(pt.coords[0])

# example: scale a line by 2 around the centroid.
line = make_line_by_length(1)
new_shape = scale(line, xfact=2, yfact=2, origin='centroid')

# example: rotate a point around origin by 45 degrees.
pt = Point([1,1])
new_pt = rotate(pt, 45, origin=[0, 0])

# example: getting object points of object0.
pts_np = get_obj_outer_pts_np('object0')
'''.strip()

prompt_f_gen_exec = '''
import numpy as np
from shapely.geometry import *
from shapely.affinity import *

# define function: total = get_total(xs=numbers).
def get_total(xs):
    return np.sum(xs)

# define function: y = eval_line(x, slope, y_intercept=0).
def eval_line(x, slope, y_intercept):
    return x * slope + y_intercept

# define function: pt = get_pt_to_the_left(pt, dist).
def get_pt_to_the_left(pt, dist):
    return pt + [-dist, 0]

# define function: pt = get_pt_to_the_top(pt, dist).
def get_pt_to_the_top(pt, dist):
    return pt + [0, dist]

# define function line = make_line_by_length(length=x).
def make_line_by_length(length):
  line = LineString([[0, 0], [length, 0]])
  return line

# define function: line = make_vertical_line_by_length(length=x).
def make_vertical_line_by_length(length):
  line = make_line_by_length(length)
  vertical_line = rotate(line, 90)
  return vertical_line

# define function: pt = interpolate_line(line, t=0.5).
def interpolate_line(line, t):
  pt = line.interpolate(t, normalized=True)
  return np.array(pt.coords[0])
'''.strip()

context_vars = {}
exec(prompt_f_gen_exec, context_vars)

# CartPole

Modified from https://colab.research.google.com/drive/1rmjdGfWsC1w-fOkICvknaGSsOUQ8Gp49

## Setup

In [ ]:
# Rendering Dependencies
!pip install gym[all]==0.21 pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
# Gym Dependencies
# !apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

# Google Colab needs to render the environment to a virtual display
# we will record this as a video and play it after the training has finished
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[-1]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## Generate Policy


In [ ]:
f_name = 'keep_pole_upright_with_pd_control'
f_sig = 'direction = keep_pole_upright_with_pd_control(x, x_dot, theta, theta_dot)'
info = 'direction is 1 if going right, 0 if going left'

policy = lmp_fgen(prompt_f_gen, f_name, f_sig, recurse=True, context_vars=context_vars, info=info)

LMP FGEN created:

# define function: direction = keep_pole_upright_with_pd_control(x, x_dot, theta, theta_dot).
# info: info: direction is 1 if going right, 0 if going left.
def keep_pole_upright_with_pd_control(x, x_dot, theta, theta_dot):
    # define constants.
    kp = 1
    kd = 1
    # define direction.
    direction = 1
    # define error.
    error = theta
    # define error_dot.
    error_dot = theta_dot
    # define control.
    control = kp * error + kd * error_dot
    # define direction.
    if control < 0:
        direction = 0
    return direction




## Run Policy

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))
observation = env.reset()
timestep = 0

while True:
  env.render()
  
  action = policy(*observation)
  
  observation, reward, done, info = env.step(action) 
  timestep += 1
  
  if done:
    break

print("run took " + str(timestep) + " timesteps")
env.close()
show_video()

run took 500 timesteps


# End-Effector Impedance Control

## Setup

In [ ]:
! pip install numpy scipy shapely astunparse pygments openai numpy-quaternion > /dev/null 2>&1
! pip install imageio==2.4.1 imageio-ffmpeg pybullet moviepy > /dev/null 2>&1

import pybullet
import pybullet_data
import cv2
from google.colab.patches import cv2_imshow
from moviepy.editor import ImageSequenceClip
import os
import itertools

# Download PyBullet assets.
if not os.path.exists('ur5e/ur5e.urdf'):
  !gdown --id 1Cc_fDSBL6QiDvNT4dpfAEbhbALSVoWcc
  !gdown --id 1yOMEm-Zp_DL3nItG9RozPeJAmeOldekX
  !gdown --id 1GsqNLhEl9dd4Mc3BM0dX3MibOI1FVWNM
  !unzip ur5e.zip
  !unzip robotiq_2f_85.zip
  !unzip bowl.zip

In [ ]:
import numpy as np
import threading
import time
import pybullet as p
import quaternion
from tqdm.auto import tqdm

class RobotEnv():

  def __init__(self):
    self.dt = 1/480
    self.sim_step = 0

    # Configure and start PyBullet.
    # python3 -m pybullet_utils.runServer
    # p.connect(p.SHARED_MEMORY)  # p.GUI for local GUI.
    p.connect(p.DIRECT)  # p.GUI for local GUI.
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    assets_path = os.path.dirname(os.path.abspath(""))
    p.setAdditionalSearchPath(assets_path)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setTimeStep(self.dt)

    self.home_joints = (np.pi / 2, -np.pi / 2, np.pi / 2, -np.pi / 2, 3 * np.pi / 2, 0)  # Joint angles: (J0, J1, J2, J3, J4, J5).
    self.home_ee_euler = (np.pi, 0, np.pi)  # (RX, RY, RZ) rotation in Euler angles.
    self.ee_link_id = 9  # Link ID of UR5 end effector.

    p.resetSimulation(p.RESET_USE_DEFORMABLE_WORLD)
    p.setGravity(0, 0, -9.8)
    self.cache_video = []

    # Temporarily disable rendering to load URDFs faster.
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 0)

    # Add robot.
    p.loadURDF("plane.urdf", [0, 0, -0.001])
    self.robot_id = p.loadURDF("ur5e/ur5e.urdf", [0, 0, 0], flags=p.URDF_USE_MATERIAL_COLORS_FROM_MTL)
    self.ghost_id = p.loadURDF("ur5e/ur5e.urdf", [0, 0, -10])  # For forward kinematics.
    self.joint_ids = [p.getJointInfo(self.robot_id, i) for i in range(p.getNumJoints(self.robot_id))]
    self.joint_ids = [j[0] for j in self.joint_ids if j[2] == p.JOINT_REVOLUTE]

    self.reset()

  def reset(self):
    # Move robot to home configuration.
    for i in range(len(self.joint_ids)):
      p.resetJointState(self.robot_id, self.joint_ids[i], self.home_joints[i])

    # Re-enable rendering.
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)

    for _ in range(200):
      p.stepSimulation()

  def get_ee_pos(self):
    ee_xyz = np.float32(p.getLinkState(self.robot_id, self.ee_link_id)[0])
    return ee_xyz

  def get_ee_lin_vel(self):
    return np.float32(p.getLinkState(self.robot_id, self.ee_link_id, computeLinkVelocity=1)[6])

  def get_ee_ang_vel(self):
      return np.float32(p.getLinkState(self.robot_id, self.ee_link_id, computeLinkVelocity=1)[7])

  def get_motor_joint_states(self):
    joint_states = p.getJointStates(self.robot_id, range(p.getNumJoints(self.robot_id)))
    joint_infos = [p.getJointInfo(self.robot_id, i) for i in range(p.getNumJoints(self.robot_id))]
    joint_states = [j for j, i in zip(joint_states, joint_infos) if i[3] > -1]
    joint_positions = [state[0] for state in joint_states]
    joint_velocities = [state[1] for state in joint_states]
    joint_torques = [state[3] for state in joint_states]
    return joint_positions, joint_velocities, joint_torques

  def get_motor_joint_idxs(self):
    joint_infos = [p.getJointInfo(self.robot_id, i) for i in range(p.getNumJoints(self.robot_id))]
    motor_joint_idxs = [idx for idx, joint_info in enumerate(joint_infos) if joint_info[3] > -1]
    return motor_joint_idxs    

  def get_jacobian(self):
    mpos, mvel, mtorq = self.get_motor_joint_states()
    result = p.getLinkState(env.robot_id,
                            env.ee_link_id,
                            computeLinkVelocity=1,
                            computeForwardKinematics=1)
    link_trn, link_rot, com_trn, com_rot, frame_pos, frame_rot, link_vt, link_vr = result

    zero_vec = [0.0] * len(mpos)
    jac_t, _ = p.calculateJacobian(env.robot_id, env.ee_link_id, com_trn, mpos, mvel, zero_vec)
    return np.array(jac_t)

  def follow_wps(self, wps, controller, K_x_mat, D_x_mat, render=False, high_res=False):
    if render:
      self.cache_video = []

    motor_joint_idxs = self.get_motor_joint_idxs()

    for x_goal in tqdm(wps):
      x_curr = self.get_ee_pos()
      x_dot = self.get_ee_lin_vel()
      J = self.get_jacobian()

      tau = controller(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J)

      p.setJointMotorControlArray(self.robot_id, motor_joint_idxs, p.TORQUE_CONTROL, forces=tau)
      self.step_sim_and_render(render=render, high_res=high_res)

  def step_sim_and_render(self, render=False, high_res=False):
    p.stepSimulation()
    self.sim_step += 1

    # Render current image at 24 FPS.
    # if self.sim_step % 60 == 0 and render:
    if self.sim_step % 20 == 0 and render:
      self.cache_video.append(self.get_camera_image(high_res=high_res))

  def get_camera_image(self, high_res):
    if not high_res:
      image_size = (240, 240)
      intrinsics = (120., 0, 120., 0, 120., 120., 0, 0, 1)
    else:
      image_size=(720, 720)
      intrinsics=(360., 0, 360., 0, 360., 360., 0, 0, 1)
    color, _, _, _, _ = self.render_image(image_size, intrinsics)
    return color

  def render_image(self, image_size=(720, 720), intrinsics=(360., 0, 360., 0, 360., 360., 0, 0, 1)):
    # Camera parameters.
    position = (0, -0.85, 0.8)
    orientation = (np.pi / 4 + np.pi / 48, np.pi, np.pi)
    orientation = p.getQuaternionFromEuler(orientation)
    zrange = (0.01, 10.)
    noise=True

    # OpenGL camera settings.
    lookdir = np.float32([0, 0, 1]).reshape(3, 1)
    updir = np.float32([0, -1, 0]).reshape(3, 1)
    rotation = p.getMatrixFromQuaternion(orientation)
    rotm = np.float32(rotation).reshape(3, 3)
    lookdir = (rotm @ lookdir).reshape(-1)
    updir = (rotm @ updir).reshape(-1)
    lookat = position + lookdir
    focal_len = intrinsics[0]
    znear, zfar = (0.01, 10.)
    viewm = p.computeViewMatrix(position, lookat, updir)
    fovh = (image_size[0] / 2) / focal_len
    fovh = 180 * np.arctan(fovh) * 2 / np.pi

    # Notes: 1) FOV is vertical FOV 2) aspect must be float
    aspect_ratio = image_size[1] / image_size[0]
    projm = p.computeProjectionMatrixFOV(fovh, aspect_ratio, znear, zfar)

    # Render with OpenGL camera settings.
    _, _, color, depth, segm = p.getCameraImage(
        width=image_size[1],
        height=image_size[0],
        viewMatrix=viewm,
        projectionMatrix=projm,
        shadow=1,
        flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX,
        renderer=p.ER_BULLET_HARDWARE_OPENGL)

    # Get color image.
    color_image_size = (image_size[0], image_size[1], 4)
    color = np.array(color, dtype=np.uint8).reshape(color_image_size)
    color = color[:, :, :3]  # remove alpha channel
    if noise:
      color = np.int32(color)
      color += np.int32(np.random.normal(0, 3, color.shape))
      color = np.uint8(np.clip(color, 0, 255))

    # Get depth image.
    depth_image_size = (image_size[0], image_size[1])
    zbuffer = np.float32(depth).reshape(depth_image_size)
    depth = (zfar + znear - (2 * zbuffer - 1) * (zfar - znear))
    depth = (2 * znear * zfar) / depth
    if noise:
      depth += np.random.normal(0, 0.003, depth.shape)

    intrinsics = np.float32(intrinsics).reshape(3, 3)
    return color, depth, position, orientation, intrinsics

  def show_video(self, loop=0):
    debug_clip = ImageSequenceClip(env.cache_video, fps=24)
    display(debug_clip.ipython_display(autoplay=1, loop=loop))

## Generate Controller

In [ ]:
f_name = 'ee_impedance_control'
f_sig = 'tau = ee_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J)'
controller = lmp_fgen(prompt_f_gen, f_name, f_sig, recurse=True, context_vars=context_vars)

LMP FGEN created:

# define function: tau = ee_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J).
def ee_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J):
    x_err = x_goal - x_curr
    x_dot_err = -x_dot
    tau = np.matmul(J.T, np.matmul(K_x_mat, x_err) + np.matmul(D_x_mat, x_dot_err))
    return tau




## Run Controller

In [ ]:
env = RobotEnv()

In [ ]:
env.reset()
init_pos = env.get_ee_pos()

K_x_mat = np.diag([5e4] * 3)
D_x_mat = np.diag([1e-3] * 3)
goal_horizon = 200

wp_goals = np.array([
    [-0.1, -0.1, 0],
    [0.2, 0, 0.1],
    [-0.1, 0.1, -0.1]
])
wps = [init_pos]
for wp_goal_pos in wp_goals:
  prev_pos = wps[-1]
  wps.extend(np.linspace(prev_pos, prev_pos + wp_goal_pos, goal_horizon).tolist())

env.follow_wps(wps, controller, K_x_mat, D_x_mat, render=True, high_res=True)
env.show_video()

  0%|          | 0/601 [00:00<?, ?it/s]

 97%|█████████▋| 30/31 [00:00<00:00, 189.10it/s]
